In [1]:
from mobfot import MobFot
from datetime import date, timedelta
from tqdm import tqdm
from joblib import Parallel, delayed
import pandas as pd

In [2]:
def generar_fechas(anio_inicio, anio_fin):
  """
  Genera una lista de fechas en formato YYYYMMDD
  entre dos años (inclusive).

  Args:
    anio_inicio (int): Año de inicio (ej. 2017)
    anio_fin (int): Año de fin (ej. 2024)

  Returns:
    lista_fechas (list): Lista de strings con las fechas.
  """
  lista_fechas = []
  fecha_inicio = date(anio_inicio, 1, 1)
  fecha_fin = date(anio_fin, 12, 31)
  while fecha_inicio <= fecha_fin:
    fecha_str = fecha_inicio.strftime("%Y%m%d")
    lista_fechas.append(fecha_str)
    fecha_inicio += timedelta(days=1)
  return lista_fechas

# Ejemplo de uso
anio_inicio = 2014
anio_fin = 2023
lista_fechas = generar_fechas(anio_inicio, anio_fin)

In [3]:
client = MobFot()

In [4]:
def day_matches(date):
    client = MobFot()
    ids = []
    day_matches = client.get_matches_by_date(date)
    for leag in range(len(day_matches["leagues"])):
        for match_leg in range(len(day_matches["leagues"][leag]["matches"])):
            ids.append(day_matches["leagues"][leag]["matches"][match_leg]["id"])
    return ids

In [5]:
workers = 12
results = Parallel(n_jobs=workers)(delayed(day_matches)(d) for d in tqdm(lista_fechas))

100%|██████████| 3652/3652 [02:46<00:00, 22.00it/s]


In [8]:
ids = []
for r in results:
    ids.extend(r)
len(ids)

512923

In [9]:
def get_info(id):
    try : 
        client = MobFot()
        info = client.get_match_details(id)
        if info["general"]["started"] == False or info["general"]["finished"] == False :
            print("Partido Cancelado")
            return False
        data = {
            "id": [id],
            'date': [info["general"]["matchTimeUTCDate"][:10]],
            'leagueName': [info["general"]["leagueName"]],
            'homeTeam': [info["general"]["homeTeam"]["name"]],
            'awayTeam': [info["general"]["awayTeam"]["name"]],
            'homeTeam_score': [info["header"]["teams"][0]["score"]],
            'awayTeam_score': [info["header"]["teams"][1]["score"]],
            'homeIdTeam': [info["general"]["homeTeam"]["id"]],
            'awayIdTeam': [info["general"]["awayTeam"]["id"]],
            'parentLeagueId': [info["general"]["parentLeagueId"]],
            'leagueId': [info["general"]["leagueId"]],
        }
        if info["content"]["stats"] != None:
            for i in range(len(info["content"]["stats"]["Periods"]["All"]["stats"])):
                for j in range(len(info["content"]["stats"]["Periods"]["All"]["stats"][i]["stats"])):
                    key = info["content"]["stats"]["Periods"]["All"]["stats"][i]["stats"][j]["key"]
                    data[key+"_home"] = info["content"]["stats"]["Periods"]["All"]["stats"][i]["stats"][j]["stats"][0]
                    data[key+"_away"] = info["content"]["stats"]["Periods"]["All"]["stats"][i]["stats"][j]["stats"][1]
    
        df_id = pd.DataFrame(data)
        return df_id
    except Exception as error:
        print(error)
        return False

In [10]:
# La primera vez, hay que crear un data frame de 0 y la busqueda con todos los 
# df = pd.DataFrame()
# filtro = ids
# Después leer el archivo guardado, y ver que ids faltan.
df = pd.read_csv('datos_fotmob_completo.csv')
filtro = list(set(ids)-set(df["id"]))
workers = 8
trabajo_por_division = int(len(filtro)/1000)
for div in tqdm(range(1, trabajo_por_division+1)):
    results = Parallel(n_jobs=workers)(delayed(get_info)(id)
                                       for id in filtro[1000*(div-1):1000*div])
    for i in results:
        if str(type(i)) == "<class 'pandas.core.frame.DataFrame'>":
            df = pd.concat([df, i], ignore_index=True)
    df.to_csv('datos_fotmob_completo.csv', header=True, index=False)

  5%|▌         | 18/328 [4:30:19<77:06:51, 895.52s/it]C:\Users\gfuen\AppData\Local\Temp\ipykernel_3128\3390480921.py:14: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, i], ignore_index=True)
 16%|█▌        | 53/328 [13:52:01<71:57:08, 941.92s/it] 


KeyboardInterrupt: 

In [ ]:
df_sin_duplicados = df.drop_duplicates()
df_sin_duplicados = df_sin_duplicados.sort_values(by="date",ignore_index=True)
# Guardar la primera corrida de datos scrapeados
df_sin_duplicados.to_csv('datos_fotmob_primer_filtro.csv', header=True, index=False)